# Reading `NIS_2022_Core.ASC`
## Understanding the `NIS 2022 Core` File Format

The `NIS_2022_Core.ASC` file is a **fixed-width ASCII data file** is a part of the Healthcare Cost and Utilization Project (HCUP). Each row in this file represents a hospital discharge record, with hundreds of variables recorded in a fixed-width layout (i.e., specific byte positions for each column, without delimiters such as a comma `,`); this makes reading the file somewhat challenging. This is why we need a text file I downloaded from [NIS_2022_Core.TXT](https://hcup-us.ahrq.gov/db/nation/nis/tools/stats/FileSpecifications_NIS_2022_Core.TXT)

### Role of the `NIS_2022_Core.TXT` File

The downloaded text file as a part of [NIS_2022_Core.TXT](https://hcup-us.ahrq.gov/db/nation/nis/nisdde.jsp) website, serves as a **file specification document**. It defines:

 - The **name** of each variable (e.g., `AGE`, `DIED`, `DRG`, etc.),
 - Its **starting and ending byte positions** in each row of the `.ASC` file,
 - Its **data type** (numeric or character),
 - A **short description** of the variable's meaning.

This metadata is essential because, as I mentioned in the meeting, the `.ASC` file does **not** contain headers or delimiters. Without the `.TXT` spec file, it would be impossible to correctly interpret the position, name, and meaning of each field.


### Best way I was able to read the Core dataset

- Have both `.TXT` and `.ASC` files along with your python notebook in the same folder.
- Extract the variable names and byte positions from the `.TXT` file
- Use `pandas.read_fwf()` to read the `.ASC` file with the correct column widths and headers.

The above approach ensures that each data element is properly aligned and labeled within the resulting DataFrame.

*Here is the code*

In [27]:
## Running this cell should take only a fraction of a second

import pandas as pd

## Note: considering the file format, you need the attached TXT file (I downloaded it online)
layout_path = "FileSpecifications_NIS_2022_Core.txt"
asc_path = "NIS_2022_Core.ASC"  

## Extract column names and byte ranges
colspecs = []
column_names = []

with open(layout_path, "r") as f:
    for line in f:
        if "NIS_2022_Core" in line:
            try:
                start = int(line[71:75].strip())
                end = int(line[76:80].strip())
                name = line[41:70].strip()
                colspecs.append((start - 1, end))
                column_names.append(name)
            except:
                continue

## Above should read the text file and understand its structure.
## In the next cell, I read the NIS_2022_Core.ASC file contents

In [31]:
## Running this cell will take a long time (about 5 minutes on my local computer)

## Use below line to read only first few 100 rows
## df = pd.read_fwf(asc_path, colspecs=colspecs, names=column_names, nrows=100)

## reading the whole dataset and loading it to DataFrame called df
df = pd.read_fwf(asc_path, colspecs=colspecs, names=column_names)

## Some initial data sanity check

In [35]:
print(f'Number of columns: {len(df.columns.to_list())}')

Number of columns: 127


In [33]:
## Check columns list
print(df.columns.to_list())

['AGE', 'AGE_NEONATE', 'AMONTH', 'AWEEKEND', 'DIED', 'DISCWT', 'DISPUNIFORM', 'DQTR', 'DRG', 'DRGVER', 'DRG_NoPOA', 'ELECTIVE', 'FEMALE', 'HCUP_ED', 'HOSP_DIVISION', 'HOSP_NIS', 'I10_BIRTH', 'I10_DELIVERY', 'I10_DX1', 'I10_DX2', 'I10_DX3', 'I10_DX4', 'I10_DX5', 'I10_DX6', 'I10_DX7', 'I10_DX8', 'I10_DX9', 'I10_DX10', 'I10_DX11', 'I10_DX12', 'I10_DX13', 'I10_DX14', 'I10_DX15', 'I10_DX16', 'I10_DX17', 'I10_DX18', 'I10_DX19', 'I10_DX20', 'I10_DX21', 'I10_DX22', 'I10_DX23', 'I10_DX24', 'I10_DX25', 'I10_DX26', 'I10_DX27', 'I10_DX28', 'I10_DX29', 'I10_DX30', 'I10_DX31', 'I10_DX32', 'I10_DX33', 'I10_DX34', 'I10_DX35', 'I10_DX36', 'I10_DX37', 'I10_DX38', 'I10_DX39', 'I10_DX40', 'I10_INJURY', 'I10_MULTINJURY', 'I10_NDX', 'I10_NPR', 'I10_PR1', 'I10_PR2', 'I10_PR3', 'I10_PR4', 'I10_PR5', 'I10_PR6', 'I10_PR7', 'I10_PR8', 'I10_PR9', 'I10_PR10', 'I10_PR11', 'I10_PR12', 'I10_PR13', 'I10_PR14', 'I10_PR15', 'I10_PR16', 'I10_PR17', 'I10_PR18', 'I10_PR19', 'I10_PR20', 'I10_PR21', 'I10_PR22', 'I10_PR23', '

In [36]:
print(df.AGE.max())  ## checking possible unusual numbers

90


In [37]:
print(df.AGE.min())  ## checking possible unusual numbers

## -99 could be a code for missing data or any particular category. Needs to be checked with data dictionary

-99


In [38]:
df.AMONTH.max()  ## maximum age month

12

In [40]:
df.DQTR.max()  ## maximum death month

4

In [42]:
df.DQTR.mean()  ## makes sense that average death quarter is around 2.5 which is (1+4)/2

2.509499310771723

In [43]:
df.FEMALE.unique()  ## possible values in column of FEMALE

## Indicator of Sex. 
# The code 0 means Male
# The code 1 means Female
# Meaning of codes -9, -6, and -8 needs to be checked with the Data Dictionary


array([ 0,  1, -9, -6, -8], dtype=int64)

In [48]:
df.FEMALE.mean() * 100  # percentage of FEMALE=1 rows. If this number was very close to 0 or 1 , it was alarming.


55.18278686580813

In [52]:
pd.set_option("display.precision", 2)

## summary statistics of the numerical columns
df.describe()

,AGE,AGE_NEONATE,AMONTH,AWEEKEND,DIED,DISCWT,DISPUNIFORM,DQTR,DRG,DRGVER,...,PRDAY22,PRDAY23,PRDAY24,PRDAY25,RACE,TOTCHG,TRAN_IN,TRAN_OUT,YEAR,ZIPINC_QRTL
count,6.58e+06,6.58e+06,6.58e+06,6.58e+06,6.58e+06,6.58e+06,6.58e+06,6.58e+06,6.58e+06,6.58e+06,...,6.58e+06,6.58e+06,6.58e+06,6.58e+06,6.58e+06,6.58e+06,6.58e+06,6.58e+06,6.58e+06,6.58e+06
mean,5.00e+01,-7.83e+00,6.56e+00,2.16e-01,2.39e-02,5.00e+00,2.83e+00,2.51e+00,5.51e+02,3.93e+01,...,-9.88e+01,-9.88e+01,-9.89e+01,-9.89e+01,1.37e+00,-7.71e+06,8.94e-02,2.84e-01,2.02e+03,2.21e+00
std,2.78e+01,3.21e+00,3.50e+00,4.13e-01,2.18e-01,1.44e-04,3.70e+00,1.20e+00,2.82e+02,4.37e-01,...,4.70e+00,4.36e+00,4.10e+00,3.86e+00,2.31e+00,8.77e+07,8.34e-01,7.04e-01,0.00e+00,1.71e+00
min,-9.90e+01,-9.00e+00,-9.00e+00,-9.00e+00,-9.00e+00,4.97e+00,-9.00e+00,-9.00e+00,1.00e+00,3.90e+01,...,-9.90e+01,-9.90e+01,-9.90e+01,-9.90e+01,-9.00e+00,-1.00e+09,-9.00e+00,-9.00e+00,2.02e+03,-9.00e+00
25%,3.00e+01,-9.00e+00,4.00e+00,0.00e+00,0.00e+00,5.00e+00,1.00e+00,2.00e+00,2.91e+02,3.90e+01,...,-9.90e+01,-9.90e+01,-9.90e+01,-9.90e+01,1.00e+00,1.70e+04,0.00e+00,0.00e+00,2.02e+03,1.00e+00
50%,5.70e+01,-9.00e+00,7.00e+00,0.00e+00,0.00e+00,5.00e+00,1.00e+00,3.00e+00,6.21e+02,3.90e+01,...,-9.90e+01,-9.90e+01,-9.90e+01,-9.90e+01,1.00e+00,3.58e+04,0.00e+00,0.00e+00,2.02e+03,2.00e+00
75%,7.30e+01,-9.00e+00,1.00e+01,0.00e+00,0.00e+00,5.00e+00,5.00e+00,4.00e+00,8.06e+02,4.00e+01,...,-9.90e+01,-9.90e+01,-9.90e+01,-9.90e+01,2.00e+00,7.45e+04,0.00e+00,0.00e+00,2.02e+03,3.00e+00
max,9.00e+01,1.00e+00,1.20e+01,1.00e+00,1.00e+00,5.01e+00,9.90e+01,4.00e+00,9.99e+02,4.00e+01,...,3.47e+02,3.33e+02,3.51e+02,3.51e+02,6.00e+00,1.00e+07,2.00e+00,2.00e+00,2.02e+03,4.00e+00


In [ ]:
## to be continued
